In [1]:
import pandas as pd

from models import evaluate_model, stan_init,\
load_prophet_model, load_scaler\
,get_model_metrics, model_add_observations, get_prophet_predictions,\
X_train_add_observations, get_y_test_new, get_EXPO_model

from plots import plot_forecast_measure
from utils import get_ETRete_clear

from statsmodels.iolib.smpickle import load_pickle

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
# sc = load_scaler('models/20210608161324_std_scaler.bin')

In [3]:
# model = load_pickle("models/20210609141628_EXPSMTH.pickle")

In [4]:
%%time
path = "data/PotenzaTermicaOraria.csv"
df = get_ETRete_clear(path)

Wall time: 23.5 s


In [5]:
col = df.columns[0]

In [6]:
df_train = df[[col]][:"2019-09"]
df_test = df[[col]]["2019-10":]

In [7]:
window = 48

In [9]:
# y_true = df_test[col].iloc[:window].values
# # y_pred = sc.inverse_transform(model.forecast(window).values.reshape(-1,1)).flatten()
# # y_pred = sc.inverse_transform(model.forecast(window).values.reshape(-1,1)
# y_pred = model.forecast(window).values

In [10]:
# evaluate_model(y_true, y_pred)

In [11]:
int(df_test.shape[0]/window)

137

In [14]:
def get_expo_evaluations_2(df_train, df_test, window=48, step_forward=24, scaler=None, verbose=True):
    
    d = {}
    for i in range(int(df_test.shape[0]/window)):
        end_window=i*step_forward
        if i==0:
            model = get_EXPO_model(df_train)
            y_true = df_test[col].iloc[:window].values
        else:
            X_train_new = X_train_add_observations(df_train, df_test[[col]].iloc[:end_window], scaler=scaler)
            model = get_EXPO_model(X_train_new)
            y_true = get_y_test_new(X_train_new, df_test, periods=window).flatten()
        
        if scaler is None:
            y_pred = model.forecast(window).values
        else:
            y_pred = scaler.inverse_transform(model.forecast(window).values.reshape(-1,1)).flatten()
            
            
            
        mae, mape = get_model_metrics(y_true, y_pred)
        d[i] = {"mae": mae, "mape": mape}
        if verbose==True:
            print("step: ", i)
#             if i>0:
#                 print(X_train_new)
#             print(y_true, y_pred)
            print("mae: ", mae,"\t mape: ", mape)

    return d

In [15]:
d2 = get_expo_evaluations_2(df_train, df_test, window=48, step_forward=24, scaler=None)

step:  0
mae:  1.7709797985723252 	 mape:  4.021243749153093
step:  1
mae:  2.3161099253581146 	 mape:  5.270883398518433
step:  2
mae:  5.060417530096937 	 mape:  10.231725851634122
step:  3
mae:  3.1408587908622274 	 mape:  6.237700397426344
step:  4
mae:  4.100141380937888 	 mape:  8.810760035105082
step:  5
mae:  4.377875519690818 	 mape:  8.916815903099932
step:  6
mae:  4.4393892220556745 	 mape:  9.041423714685882
step:  7
mae:  5.2930565788936965 	 mape:  11.004227537840439
step:  8
mae:  3.820705698356076 	 mape:  6.859413698932683
step:  9
mae:  9.328222182952345 	 mape:  17.66994555224445
step:  10
mae:  4.21145110193265 	 mape:  7.686399242446644
step:  11
mae:  4.058413517193516 	 mape:  7.319869820995871
step:  12
mae:  5.022138894631246 	 mape:  8.841713648177173
step:  13
mae:  19.436210246849836 	 mape:  26.096987151332602
step:  14
mae:  20.277268540399646 	 mape:  21.451319683355372
step:  15
mae:  17.497659884464095 	 mape:  18.680039596613717
step:  16
mae:  11.671

In [16]:
pd.DataFrame().from_dict(d2, orient="index").mean()

mae     22.756544
mape    10.646500
dtype: float64